for loop을 돌 때, loop 안에서 정의된 변수값에 대해서 처리가 끝나면 그 변수는 사용하지 않습니다. 

    for item in list:
        # do something with item
        
특히 토크나이징을 하는 작업 등은 text 파일의 line by line으로 처리해도 좋습니다. 메모리에 올리지 않아도 됩니다. 이처럼 generator 형태로 text를 메모리에 line by line으로 올려서 작업을 하려면 \_\_iter\_\_ 함수를 구현하면 됩니다. 우리가 이용하는 DoublespaceLineCorpus는 이미 iter가 구현되어 있습니다. 

In [1]:
class DoublespaceLineCorpus:    
    def __init__(self, corpus_fname, iter_sent = False):
        self.corpus_fname = corpus_fname
        self.iter_sent = iter_sent

    def __iter__(self):
        with open(self.corpus_fname, encoding='utf-8') as f:
            for doc_idx, doc in enumerate(f):
                if not self.iter_sent:
                    yield doc
                    continue
                for sent in doc.split('  '):                    
                    sent = sent.strip()
                    if not sent: continue
                    yield sent

corpus = DoublespaceLineCorpus('./texts.txt', iter_sent=True)
for n_sent, sent in enumerate(corpus):
    print('{} sent: {}. its length is {}'.format(n_sent, sent, len(sent)))

0 sent: 테스트 데이터 입니다. its length is 11
1 sent: 이것만 한 줄에 두 개의 문장을 넣어둘 겁니다. its length is 25
2 sent: 문장을 이렇게 만들어 둘거에요. its length is 16
3 sent: yield test와. its length is 11
4 sent: len test를 해보세요. its length is 14


위의 예시에서 메모리에는 for loop에서 정의되는 sent만 만들어져 사용되고, 다음 loop으로 넘어가면서 메모리에 올라가지 않습니다. 

    for n_sent, sent in enumerate(corpus):
        print('{} sent: {}. its length is {}'.format(n_sent, sent, len(sent)))
        
이처럼 generator를 만들기 위해서는 iter 함수 내에서 yield를 이용해야 합니다. 

     for sent in doc.split('  '):                    
        sent = sent.strip()
        if not sent: continue
        yield sent
        
위 부분은 doc 의 sent에 대하여 empty str이 아니면 아래의 "for sent in corpus" 부분에 sent로 메모리에 그 str 값을 올리라는 의미입니다. 

    for n_sent, sent in enumerate(corpus):

str은 length가 있습니다. list나 set도 length가 있습니다. 이는 len(object) 형식으로 이용할 수 있습니다. 

In [2]:
s = '12345'
len(s) # 5

5

하지만 아직 DoublespaceLineCorpus 는 length를 구할 수가 없습니다. 텍스트가 4줄이 있으니 len(corpus)를 하면 4가 출력되도록 만들고 싶습니다. 

In [3]:
len(corpus)

TypeError: object of type 'DoublespaceLineCorpus' has no len()

\_\_iter\_\_을 구현한 것처럼 \_\_len\_\_을 구현하면 이를 할 수 있습니다. 단 우리는 iter_sent를 True와 False로 만들 수 있기 때문에 sentence length와 document length를 모두 구할 수 있도록 만들어 보겠습니다. 

In [4]:
class DoublespaceLineCorpus:    
    def __init__(self, corpus_fname, iter_sent = False):
        self.corpus_fname = corpus_fname
        self.iter_sent = iter_sent
        self.num_sents = 0
        self.num_docs = 0

    def __iter__(self):
        with open(self.corpus_fname, encoding='utf-8') as f:
            for doc_idx, doc in enumerate(f):
                if not self.iter_sent:
                    yield doc
                    continue
                for sent in doc.split('  '):                    
                    sent = sent.strip()
                    if not sent: continue
                    yield sent
    
    def __len__(self):
        if self.iter_sent:
            if self.num_sents == 0:
                with open(self.corpus_fname, encoding='utf-8') as f:
                    self.num_sents = sum((True for doc in f for sent in doc.strip().split('  ') if sent.strip()))
            return self.num_sents
        else:
            if self.num_docs == 0:
                with open(self.corpus_fname, encoding='utf-8') as f:
                    self.num_docs = sum((True for doc in f if doc.strip()))
            return self.num_docs

In [5]:
corpus = DoublespaceLineCorpus('texts.txt', iter_sent=True)
print('when iter_sent == True, n sents = {}'.format(len(corpus)))

corpus.iter_sent=False
print('when iter_sent == False, n docs = {}'.format(len(corpus)))

when iter_sent == True, n sents = 5
when iter_sent == False, n docs = 4
